In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import raytrace as rt
import numpy as np
import math
from pylab import *
from raytrace import implane
import rtcore

/home/hp/miniconda3/envs/py27/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [3]:
freq_list = [10.e6, 18.e6, 40.e6, 80.e6, 200.e6, 3000.e6] # Hz, radio wave frequency
grid=(100,100)
rect=(-1, -1, 1, 1)
obs=(215, 0, 0)  # Position of the observer (the earth's coordinates) in SGI system
rsph = 10  # Radius of the integrarion sphere
niter = 1500  #The maximum iterations to perform using the algorithm

In [ ]:
nx = int(grid[0])
ny = int(grid[1])

# Y-Z image plane coordinates
ymin, ymax = -1.0, 1.0
zmin, zmax = -1.0, 1.0

y = np.linspace(ymin, ymax, nx)
z = np.linspace(zmin, zmax, ny)

# ---------------------------------------------------
# SELECT SINGLE RAY LOCATION
# ---------------------------------------------------
# choose y ~ 0 (center)
iy = np.argmin(np.abs(y - 0.0))

# choose z = 0.18 solar radii
iz = np.argmin(np.abs(z - 0.18))

# trkrays must be list of [iy, iz]
trkrays = [[iy, iz]]

print('Tracking grid index:', trkrays)
print('Physical location (Y,Z)=', y[iy], z[iz])

# ---------------------------------------------------
# 3D PLOT SETUP
# ---------------------------------------------------
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

# ---------------------------------------------------
# LOOP OVER FREQUENCIES
# ---------------------------------------------------
for freq_hz in freq_list:

    # build implane object
    test1 = rt.implane(grid,
                       rect,
                       obs,
                       rsph,
                       freq=freq_hz,
                       mode='TbrIQUV',
                       trkparms=['pos'],
                       trknpmax=1500)

    test1.package = '/home/hp/raytrace/py_raytr_threaded'

    # trace SINGLE ray
    test1.trace(niter, trkrays)

    traj = test1.traj.pos

    # plot trajectory
    ax.plot(traj[0,:,0],   # X
            traj[0,:,1],   # Y
            traj[0,:,2],   # Z
            linewidth=2,
            label='%.0f MHz' % (freq_hz/1e6))

# ---------------------------------------------------
# AXIS SETTINGS
# ---------------------------------------------------
ax.set_xlabel('X (Solar Radii)')
ax.set_ylabel('Y (Solar Radii)')
ax.set_zlabel('Z (Solar Radii)')

ax.set_xlim(-6,6)
ax.set_ylim(-6,6)
ax.set_zlim(-6,6)

ax.legend()

plt.show()
